# Webcam Application

In [1]:
!pip install -U PyObjC
!pip install playsound

In [3]:
import cv2 
import numpy as np 
from playsound import playsound 
from PIL import Image, ImageDraw
import face_recognition 
from tensorflow import keras

### Loading the Classifier Model

In [4]:
eye_model = keras.models.load_model('models/drowsiness-detection-cnn-model.h5')

Metal device set to: Apple M1


### Capture the Eye

In [6]:
def eye_cropper(frame):
    """
    Detects the eye area in a given frame, crops it, 
    and resizes it to a fixed size for prediction.

    Args:
        frame: a NumPy array representing a color image

    Returns:
        A NumPy array of shape (1, 80, 80, 3) representing 
        the cropped and resized eye area for prediction. 
        Returns None if no eye is found in the given frame.
    """

    # capture facial landmarks
    facial_features_list = face_recognition.face_landmarks(frame)


    # capture eye cordinates
    try:
        eye = facial_features_list[0]['left_eye']
    except:
        try:
            eye = facial_features_list[0]['right_eye']
        except:
            return

    # establish the max x and y coordinates of the eye
    x_max = max([coordinate[0] for coordinate in eye])
    x_min = min([coordinate[0] for coordinate in eye])
    y_max = max([coordinate[1] for coordinate in eye])
    y_min = min([coordinate[1] for coordinate in eye])

    x_range = x_max - x_min
    y_range = y_max - y_min

    # add 50% cushion
    if x_range > y_range:
        right = round(.5*x_range) + x_max
        left = x_min - round(.5*x_range)
        bottom = round((((right-left) - y_range))/2) + y_max
        top = y_min - round((((right-left) - y_range))/2)
    else:
        bottom = round(.5*y_range) + y_max
        top = y_min - round(.5*y_range)
        right = round((((bottom-top) - x_range))/2) + x_max
        left = x_min - round((((bottom-top) - x_range))/2)


    # crop the image according to the coordinates determined above
    cropped = frame[top:(bottom + 1), left:(right + 1)]

    # resize the image
    cropped = cv2.resize(cropped, (80,80))
    image_for_prediction = cropped.reshape(-1, 80, 80, 3)

    return image_for_prediction

The following function takes an image frame and marks the facial landmarks on the image.

In [10]:
def mark_facial_landmarks(frame):
    """"
    The function mark_facial_landmarks takes in an image frame and marks 
    the facial landmarks on the image with green circles.

    Args:
        frame: A numpy array representing an image.
    """
    face_landmarks_list = face_recognition.face_landmarks(frame)
    for face_landmarks in face_landmarks_list:
        # Trace out each facial feature in the image with a line
        for facial_feature in face_landmarks.keys():
            if facial_feature == 'left_eye' or facial_feature == 'right_eye':
                for cord in face_landmarks[facial_feature]:
                    cv2.circle(frame, cord, radius=1, color=(0,255,0), thickness=2)

The following function shows the state if the eye on the given frame

In [11]:
def show_eye_state(image, text):
    """
    Shows the state of the eye on the input image with a text annotation.

    Args:
        image: numpy.ndarray. The input image to annotate.
        text: str. The text to annotate the image with.
    """

    # split text string into separate lines
    lines = text.split("\n")

    # define starting position of text
    pos = (50, 50)

    # loop over each line of text and draw on image
    for line in lines:
        cv2.putText(image, line, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 1)
        pos = (pos[0], pos[1]+25)

The following function modifies the input image directly by adding bounding boxes around the faces.

In [13]:
def draw_bounding_box(image):
    """
    Takes an image as input and draws a green bounding box 
    around each face in the image. It uses the face_recognition 
    library to locate the faces in the image. 
    The function modifies the input image directly.

    Args:
        image: A numpy array representing an image in BGR format.
    """
    
    # Find the face locations in the image
    face_locations = face_recognition.face_locations(image)

    # Draw a rectangle around each face
    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(image, (left, top), (right, bottom), (0,255,0), 1)

In [15]:
# initiate webcam
cap = cv2.VideoCapture(0)
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

if not cap.isOpened():
    raise IOError('Cannot open webcam')

In [16]:
counter = 0

# create a while loop that runs while webcam is in use
while True:

    # capture frames being outputted by webcam
    ret, frame = cap.read()

    # use only every other frame to manage speed and memory usage
    frame_count = 0
    if frame_count == 0:
        frame_count += 1
        pass
    else:
        count = 0
        continue

    # capture and crop the eye for prediction
    image_for_prediction = eye_cropper(frame)
    try:
        image_for_prediction = image_for_prediction/255.0
    except:
        continue

    # get prediction from model
    prediction = eye_model.predict(image_for_prediction)

    # display information on the frame based on the prediction
    if prediction < 0.5:
        counter = 0
        status = 'Open'
        
        # mark eye cordinated on the frame
        mark_facial_landmarks(frame)
        
        # draw the bounding box on the frame
        draw_bounding_box(frame)
        
        # display information related to eye state on the frame
        message = 'Frames per second: '+ str(fps) + '\nEye State: ' + status + '\nDrowsiness: ' + str(round(prediction[0][0], 4))
        show_eye_state(frame, message)
        
    else:
        counter = counter + 1
        status = 'Closed'
        
        # mark eye cordinated on the frame
        mark_facial_landmarks(frame)
        
        # draw the bounding box on the frame
        draw_bounding_box(frame)
        
        # display information related to eye state on the frame
        message = 'Frames per second: '+ str(fps) + '\nEye State: ' + status + '\nEye Closure Time: ' + str(counter) + ' sec' + '\nDrowsiness: ' + str(round(prediction[0][0], 4))
        show_eye_state(frame, message)

        # alert the user if the counter exceeds 2 seconds
        if counter > 2:
            pos = (50, 175)
            cv2.putText(frame, 'Drowsiness Alert!', pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            cv2.imshow('Drowsiness Detection', frame)
            k = cv2.waitKey(1)
            
            playsound('rooster.mov')
            continue
            
    cv2.imshow('Drowsiness Detection', frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

2023-05-12 09:46:20.409846: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 10ms/step


KeyboardInterrupt: 